In [1]:
import sys
from pathlib import Path
import nest_asyncio
import asyncio
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone  # Added timezone here

# Enable async support in Jupyter
nest_asyncio.apply()

# Add the project root directory to Python path
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to Python path")

# Now we can safely import our local modules
from sqlalchemy import select, func
from database.session import get_db_session
from database.models import WeatherForecast, WeatherObservation

print("Setup complete!")

Added /Users/leekorff-korn/Code/weather-man to Python path
Setup complete!


In [2]:
async def load_data(start_date=None, end_date=None, location="NYC_CASTLE_2"):
    """Load forecasts and observations from database."""
    async with get_db_session() as session:
        # Base query for forecasts with location and daytime filter
        forecast_query = select(WeatherForecast).where(
            WeatherForecast.location == location,
            WeatherForecast.is_daytime == True  # Filter for daytime forecasts
        )
        if start_date:
            forecast_query = forecast_query.where(WeatherForecast.forecast_time >= start_date)
        if end_date:
            forecast_query = forecast_query.where(WeatherForecast.forecast_time <= end_date)
        
        result = await session.execute(forecast_query)
        forecasts = result.scalars().all()
        print(f"DEBUG: Found {len(forecasts)} daytime forecasts for {location}")
        
        if len(forecasts) > 0:
            forecast_times = {f.forecast_time for f in forecasts}
            min_time = min(forecast_times)
            max_time = max(forecast_times) + timedelta(hours=1)
            
            # For observations, we'll match the times from our daytime forecasts
            obs_query = select(WeatherObservation).where(
                WeatherObservation.location == location,
                WeatherObservation.observed_time >= min_time,
                WeatherObservation.observed_time <= max_time,
                # We don't need a daytime filter here since we're matching forecast times
            )
            result = await session.execute(obs_query)
            observations = result.scalars().all()
            print(f"DEBUG: Found {len(observations)} matching observations for {location}")
        else:
            observations = []
            
        return forecasts, observations

# For the visualization, replace the matplotlib code in analyze_forecast_accuracy with:

# Try loading without date filters first
print("DEBUG: Testing database connection without date filters...")
forecasts, observations = await load_data()

if len(forecasts) == 0:
    print("\nDEBUG: No data found. Possible issues:")
    print("1. Database connection issues")
    print("2. Empty tables")
    print("3. Timezone mismatch in queries")
    print("\nTrying with explicit date range...")
    
    # Now try with date filters
    start_date = datetime(2024, 1, 1, tzinfo=timezone.utc)
    end_date = datetime(2024, 2, 1, tzinfo=timezone.utc)
    forecasts, observations = await load_data(start_date, end_date)

DEBUG: Testing database connection without date filters...
DEBUG: Found 624 daytime forecasts for NYC_CASTLE_2
DEBUG: Found 44 matching observations for NYC_CASTLE_2


In [3]:
async def load_daytime_forecast_data(start_date=None, end_date=None, location="NYC_CASTLE_2"):
    """
    Load forecasts created during daytime hours and their corresponding observations.
    Specifically looks at forecasts created within 12 hours before the day's high temperature.
    """
    async with get_db_session() as session:
        # First, get all observations to find daily highs
        obs_query = select(WeatherObservation).where(
            WeatherObservation.location == location
        )
        if start_date:
            obs_query = obs_query.where(WeatherObservation.observed_time >= start_date)
        if end_date:
            obs_query = obs_query.where(WeatherObservation.observed_time <= end_date)
        
        result = await session.execute(obs_query)
        observations = result.scalars().all()
        
        if not observations:
            print("No observations found")
            return [], []
        
        # Find daily high temperatures and their times
        daily_highs = {}  # {date: (max_temp, observation_time)}
        for obs in observations:
            date = obs.observed_time.date()
            if date not in daily_highs or obs.temperature > daily_highs[date][0]:
                daily_highs[date] = (obs.temperature, obs.observed_time)
            elif obs.temperature == daily_highs[date][0] and obs.observed_time > daily_highs[date][1]:
                # If same temperature but later in the day, use the later time
                daily_highs[date] = (obs.temperature, obs.observed_time)
        
        # Now get forecasts created within 12 hours before each daily high
        relevant_forecasts = []
        for date, (max_temp, high_time) in daily_highs.items():
            # Calculate the start of the window (12 hours before the high)
            window_start = high_time - timedelta(hours=12)
            
            # Get forecasts created in this window
            forecast_query = select(WeatherForecast).where(
                WeatherForecast.location == location,
                WeatherForecast.created_at >= window_start,
                WeatherForecast.created_at <= high_time
            )
            
            result = await session.execute(forecast_query)
            day_forecasts = result.scalars().all()
            relevant_forecasts.extend(day_forecasts)
        
        print(f"DEBUG: Found {len(relevant_forecasts)} forecasts created within 12 hours "
              f"of daily highs for {location}")
        print(f"DEBUG: Analyzed {len(daily_highs)} days with high temperatures")
        
        # Return both the filtered forecasts and all observations
        # (we need all observations for context)
        return relevant_forecasts, observations

# Example usage:
print("DEBUG: Loading forecasts created during daytime hours...")
forecasts, observations = await load_daytime_forecast_data()

if len(forecasts) == 0:
    print("\nDEBUG: No data found. Possible issues:")
    print("1. Database connection issues")
    print("2. Empty tables")
    print("3. No forecasts within time windows")
    print("\nTrying with explicit date range...")
    
    # Try with date filters
    start_date = datetime(2024, 1, 1, tzinfo=timezone.utc)
    end_date = datetime(2024, 2, 1, tzinfo=timezone.utc)
    forecasts, observations = await load_daytime_forecast_data(start_date, end_date)

DEBUG: Loading forecasts created during daytime hours...
DEBUG: Found 552 forecasts created within 12 hours of daily highs for NYC_CASTLE_2
DEBUG: Analyzed 3 days with high temperatures


In [4]:
def process_data(forecasts, observations):
    """Convert data to pandas DataFrames and process for analysis."""
    # Convert forecasts to DataFrame
    forecast_data = []
    for f in forecasts:
        # Round the forecast time to the nearest hour
        forecast_time = f.forecast_time.replace(minute=0, second=0, microsecond=0)
        forecast_data.append({
            'forecast_time': f.forecast_time,
            'created_at': f.created_at,
            'location': f.location,
            'temperature': f.temperature,
            'forecast_time_hour': forecast_time  # Changed from forecast_hour
        })
    df_forecasts = pd.DataFrame(forecast_data)
    
    # Convert observations to DataFrame
    obs_data = []
    for o in observations:
        # Round the observation time to the nearest hour
        observation_time = o.observed_time.replace(minute=0, second=0, microsecond=0)
        obs_data.append({
            'observed_time': o.observed_time,
            'location': o.location,
            'temperature': o.temperature,
            'observation_time_hour': observation_time  # Changed from observation_hour
        })
    df_observations = pd.DataFrame(obs_data)
    
    # Group forecasts by forecast hour
    grouped_forecasts = df_forecasts.groupby('forecast_time_hour')
    
    return df_forecasts, df_observations, grouped_forecasts

In [5]:
def process_data(forecasts, observations):
    """Convert data to pandas DataFrames and process for analysis."""
    # Convert forecasts to DataFrame
    forecast_data = []
    for f in forecasts:
        # Round the forecast time to the nearest hour and convert to EST
        forecast_time = f.forecast_time.replace(minute=0, second=0, microsecond=0)
        forecast_time_est = forecast_time.astimezone(timezone(timedelta(hours=-5)))
        created_at_est = f.created_at.astimezone(timezone(timedelta(hours=-5)))
        
        forecast_data.append({
            'forecast_time': forecast_time_est,  # Now in EST
            'created_at': created_at_est,        # Now in EST
            'location': f.location,
            'temperature': f.temperature,
            'forecast_time_hour': forecast_time_est  # Changed from forecast_hour
        })
    df_forecasts = pd.DataFrame(forecast_data)
    
    # Debug print
    print("Forecast DataFrame columns:", df_forecasts.columns.tolist())
    print("First few rows of forecasts:")
    print(df_forecasts.head())
    
    # Convert observations to DataFrame
    obs_data = []
    for o in observations:
        # Round the observation time to the nearest hour and convert to EST
        observation_time = o.observed_time.replace(minute=0, second=0, microsecond=0)
        observation_time_est = observation_time.astimezone(timezone(timedelta(hours=-5)))
        
        obs_data.append({
            'observed_time': observation_time_est,  # Now in EST
            'location': o.location,
            'temperature': o.temperature,
            'observation_time_hour': observation_time_est  # Changed from observation_hour
        })
    df_observations = pd.DataFrame(obs_data)
    
    # Debug print
    print("\nObservation DataFrame columns:", df_observations.columns.tolist())
    print("First few rows of observations:")
    print(df_observations.head())
    
    # Group forecasts by forecast hour
    grouped_forecasts = df_forecasts.groupby('forecast_time_hour')
    
    return df_forecasts, df_observations, grouped_forecasts

In [6]:

def analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts):
    """Analyze and visualize forecast accuracy."""
    accuracy_data = []
    
    for forecast_time_hour, group in grouped_forecasts:
        hour_observations = df_observations[
            (df_observations['observed_time'] >= forecast_time_hour) &
            (df_observations['observed_time'] < forecast_time_hour + timedelta(hours=1))
        ]
        
        if not hour_observations.empty:
            actual_temp = hour_observations['temperature'].mean()
            
            for _, forecast in group.iterrows():
                hours_ahead = (forecast_time_hour - forecast['created_at']).total_seconds() / 3600
                error = forecast['temperature'] - actual_temp
                accuracy_data.append({
                    'forecast_time': forecast_time_hour,
                    'hours_ahead': hours_ahead,
                    'error': error,
                    'actual_temp': actual_temp,
                    'forecast_temp': forecast['temperature'],
                    'error_magnitude': abs(error)  # For coloring
                })
    
    df_accuracy = pd.DataFrame(accuracy_data)
    
    # Create interactive plot with plotly
    import plotly.express as px
    
    fig = px.scatter(df_accuracy, 
                     x='hours_ahead',
                     y='error',
                     color='error_magnitude',
                     color_continuous_scale='RdYlBu_r',  # Red for large errors, blue for small
                     hover_data=['forecast_time', 'actual_temp', 'forecast_temp'],
                     labels={
                         'hours_ahead': 'Hours Ahead of Forecast',
                         'error': 'Forecast Error (°F)',
                         'error_magnitude': 'Absolute Error (°F)'
                     },
                     title='Forecast Error vs Forecast Lead Time')
    
    fig.add_hline(y=0, line_dash="dash", line_color="red")
    fig.update_layout(
        height=600,
        showlegend=True,
        hovermode='closest'
    )
    
    fig.show()
    return df_accuracy

In [7]:
# Run analysis and create visualization
df_forecasts, df_observations, grouped_forecasts = process_data(forecasts, observations)
print(f"\nProcessed {len(df_forecasts)} forecasts into {len(grouped_forecasts)} hourly groups")

df_accuracy = analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts)

print("Analysis complete")
print(f"Mean absolute error: {abs(df_accuracy['error']).mean():.2f}°F")

Forecast DataFrame columns: ['forecast_time', 'created_at', 'location', 'temperature', 'forecast_time_hour']
First few rows of forecasts:
              forecast_time                       created_at      location  \
0 2024-12-09 00:00:00-05:00 2024-12-08 23:00:25.697307-05:00  NYC_CASTLE_2   
1 2024-12-09 01:00:00-05:00 2024-12-08 23:00:25.697307-05:00  NYC_CASTLE_2   
2 2024-12-09 02:00:00-05:00 2024-12-08 23:00:25.697307-05:00  NYC_CASTLE_2   
3 2024-12-09 03:00:00-05:00 2024-12-08 23:00:25.697307-05:00  NYC_CASTLE_2   
4 2024-12-09 04:00:00-05:00 2024-12-08 23:00:25.697307-05:00  NYC_CASTLE_2   

   temperature        forecast_time_hour  
0         48.0 2024-12-09 00:00:00-05:00  
1         47.0 2024-12-09 01:00:00-05:00  
2         46.0 2024-12-09 02:00:00-05:00  
3         45.0 2024-12-09 03:00:00-05:00  
4         44.0 2024-12-09 04:00:00-05:00  

Observation DataFrame columns: ['observed_time', 'location', 'temperature', 'observation_time_hour']
First few rows of observations:
 

Analysis complete
Mean absolute error: 1.83°F


In [8]:
def analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts):
    """Analyze and visualize forecast accuracy from multiple perspectives."""
    accuracy_data = []
    
    for forecast_time_hour, group in grouped_forecasts:
        hour_observations = df_observations[
            (df_observations['observed_time'] >= forecast_time_hour) &
            (df_observations['observed_time'] < forecast_time_hour + timedelta(hours=1))
        ]
        
        if not hour_observations.empty:
            actual_temp = hour_observations['temperature'].mean()
            
            for _, forecast in group.iterrows():
                hours_ahead = (forecast_time_hour - forecast['created_at']).total_seconds() / 3600
                error = forecast['temperature'] - actual_temp
                accuracy_data.append({
                    'forecast_time': forecast_time_hour,
                    'hours_ahead': round(hours_ahead),  # Round to nearest hour for better grouping
                    'error': error,
                    'actual_temp': actual_temp,
                    'forecast_temp': forecast['temperature'],
                    'error_magnitude': abs(error),
                    'hour_of_day': forecast_time_hour.hour,
                    'date': forecast_time_hour.date()
                })
    
    df_accuracy = pd.DataFrame(accuracy_data)
    
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # 1. Box plot of errors by hours ahead
    fig1 = px.box(df_accuracy, 
                  x='hours_ahead',
                  y='error',
                  title='Distribution of Forecast Errors by Lead Time',
                  labels={'hours_ahead': 'Hours Ahead of Forecast',
                         'error': 'Forecast Error (°F)'})
    fig1.show()
    
    # 2. Average absolute error by hour of day

    
    # In this graph, hour_of_day comes from forecast_time_hour which is derived from forecast_time in our data processing:
    # So this graph shows the average error magnitude for each hour of the day that was being forecast for (the forecast_time), not when the forecast was made (created_at).
    #     For example, if the bar at hour 14 (2 PM EST) shows an error of 3°F, this means:
    #         For all forecasts that were predicting the temperature at 2 PM
    #         Regardless of when those forecasts were made (could be 6 hours ahead, 12 hours ahead, etc.)
    #         The average absolute error was 3°F
    
    hourly_mae = df_accuracy.groupby('hour_of_day')['error_magnitude'].mean().reset_index()
    fig2 = px.bar(hourly_mae,
                  x='hour_of_day',
                  y='error_magnitude',
                  title='Average Forecast Error by Hour of Day (EST)',
                  labels={'hour_of_day': 'Hour of Day (EST)',
                         'error_magnitude': 'Mean Absolute Error (°F)'})
    fig2.show()
    
    # 3. Heatmap of average absolute error by hour of day and hours ahead
    pivot_data = df_accuracy.pivot_table(
        values='error_magnitude',
        index='hour_of_day',
        columns='hours_ahead',
        aggfunc='mean'
    )
    
    fig3 = px.imshow(pivot_data,
                     title='Error Heatmap: Hour of Day vs Forecast Lead Time',
                     labels=dict(x='Hours Ahead', y='Hour of Day (EST)', color='Mean Absolute Error (°F)'),
                     aspect='auto',
                     color_continuous_scale='RdYlBu_r')
    fig3.show()
    
    # 4. Summary statistics
    print("\nSummary Statistics:")
    print(f"Overall Mean Absolute Error: {df_accuracy['error_magnitude'].mean():.2f}°F")
    print(f"Median Absolute Error: {df_accuracy['error_magnitude'].median():.2f}°F")
    print("\nMean Absolute Error by Lead Time:")
    print(df_accuracy.groupby('hours_ahead')['error_magnitude'].mean().round(2))
    
    # 5. Scatter plot with simple linear trendline instead of LOWESS
    fig4 = px.scatter(df_accuracy,
                      x='hours_ahead',
                      y='error_magnitude',
                      trendline="ols",  # Changed from "lowess" to "ols" (linear)
                      title='Forecast Error vs Lead Time with Trend',
                      labels={'hours_ahead': 'Hours Ahead of Forecast',
                             'error_magnitude': 'Absolute Error (°F)'},
                      opacity=0.5)
    fig4.show()
    
    return df_accuracy

# Run the analysis
df_accuracy = analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts)


Summary Statistics:
Overall Mean Absolute Error: 1.83°F
Median Absolute Error: 1.08°F

Mean Absolute Error by Lead Time:
hours_ahead
1     1.89
2     1.89
3     2.00
4     2.11
5     2.27
6     2.51
7     2.76
8     2.76
9     2.68
10    2.34
11    1.80
12    1.32
13    0.87
14    0.63
15    0.72
16    0.78
17    0.67
18    0.56
19    0.45
20    0.50
21    0.64
22    0.92
23    0.92
Name: error_magnitude, dtype: float64


In [9]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts):
    """Analyze and visualize forecast accuracy."""
    accuracy_data = []
    
    for forecast_time_hour, group in grouped_forecasts:
        hour_observations = df_observations[
            (df_observations['observed_time'] >= forecast_time_hour) &
            (df_observations['observed_time'] < forecast_time_hour + timedelta(hours=1))
        ]
        
        if not hour_observations.empty:
            actual_temp = hour_observations['temperature'].mean()
            
            for _, forecast in group.iterrows():
                hours_ahead = (forecast_time_hour - forecast['created_at']).total_seconds() / 3600
                error = forecast['temperature'] - actual_temp
                accuracy_data.append({
                    'forecast_time': forecast_time_hour,
                    'created_at': forecast['created_at'],
                    'created_hour': forecast['created_at'].hour,
                    'hours_ahead': round(hours_ahead),  # Round to nearest hour
                    'error': error,
                    'actual_temp': actual_temp,
                    'forecast_temp': forecast['temperature'],
                    'error_magnitude': abs(error)
                })
    
    df_accuracy = pd.DataFrame(accuracy_data)
    
    # 1. Average error by how many hours ahead the forecast was made
    hours_ahead_mae = df_accuracy.groupby('hours_ahead')['error_magnitude'].agg(['mean', 'count']).reset_index()
    fig1 = px.bar(hours_ahead_mae,
                  x='hours_ahead',
                  y='mean',
                  title='Average Forecast Error by Hours Ahead',
                  labels={'hours_ahead': 'Hours Ahead of Forecast Time',
                         'mean': 'Mean Absolute Error (°F)'},
                  hover_data=['count'])  # Show number of forecasts for each bar
    fig1.show()
    
    # 2. Average error by creation hour
    created_hour_mae = df_accuracy.groupby('created_hour')['error_magnitude'].mean().reset_index()
    fig2 = px.bar(created_hour_mae,
                  x='created_hour',
                  y='error_magnitude',
                  title='Average Forecast Error by Hour Created (EST)',
                  labels={'created_hour': 'Hour Forecast Was Made (EST)',
                         'error_magnitude': 'Mean Absolute Error (°F)'})
    fig2.show()
    
    # 3. Heatmap showing error by creation hour and hours ahead
    pivot_data = df_accuracy.pivot_table(
        values='error_magnitude',
        index='created_hour',
        columns='hours_ahead',
        aggfunc='mean'
    )
    
    fig3 = px.imshow(pivot_data,
                     title='Error Heatmap: Creation Hour vs Hours Ahead',
                     labels=dict(x='Hours Ahead', 
                               y='Hour Forecast Was Made (EST)', 
                               color='Mean Absolute Error (°F)'),
                     aspect='auto',
                     color_continuous_scale='RdYlBu_r')
    fig3.show()
    
    # 4. Summary statistics by forecast horizon
    print("\nSummary Statistics by Forecast Horizon:")
    horizons = [0, 6, 12, 24, 48, 72]  # Different forecast horizons to analyze
    for i in range(len(horizons)-1):
        mask = (df_accuracy['hours_ahead'] >= horizons[i]) & (df_accuracy['hours_ahead'] < horizons[i+1])
        mae = df_accuracy[mask]['error_magnitude'].mean()
        count = mask.sum()
        print(f"{horizons[i]}-{horizons[i+1]} hours ahead: {mae:.2f}°F (n={count})")
    
    return df_accuracy

# Run the analysis
df_accuracy = analyze_forecast_accuracy(df_forecasts, df_observations, grouped_forecasts)


Summary Statistics by Forecast Horizon:
0-6 hours ahead: 2.02°F (n=100)
6-12 hours ahead: 2.50°F (n=87)
12-24 hours ahead: 0.78°F (n=75)
24-48 hours ahead: nan°F (n=0)
48-72 hours ahead: nan°F (n=0)


In [10]:
def analyze_daily_max_temps(df_forecasts, df_observations):
    """Analyze forecast accuracy for daily maximum temperatures."""
    
    # First, find the actual daily max temperatures and their times
    daily_maxes = []
    
    # Group observations by date
    df_observations['date'] = df_observations['observed_time'].dt.date
    daily_obs = df_observations.groupby('date')
    
    for date, day_obs in daily_obs:
        # Find max temperature for the day
        max_temp = day_obs['temperature'].max()
        # Get all observations that match the max temp
        max_temp_times = day_obs[day_obs['temperature'] == max_temp]
        
        if not max_temp_times.empty:
            # Take the first occurrence of max temp for this day
            max_record = max_temp_times.iloc[0]
            max_hour = max_record['observed_time'].replace(minute=0, second=0, microsecond=0)
            
            # Find matching forecasts (those predicting this hour)
            matching_forecasts = df_forecasts[
                (df_forecasts['forecast_time_hour'] == max_hour)
            ]
            
            if not matching_forecasts.empty:
                # Include all forecasts for this max temp hour
                for _, forecast in matching_forecasts.iterrows():
                    error = forecast['temperature'] - max_temp
                    daily_maxes.append({
                        'date': date,
                        'max_temp_hour': max_hour,
                        'actual_max_temp': max_temp,
                        'forecast_temp': forecast['temperature'],
                        'forecast_created': forecast['created_at'],
                        'error': error,
                        'error_magnitude': abs(error),
                        'hours_ahead': (max_hour - forecast['created_at']).total_seconds() / 3600
                    })
    
    # Convert to DataFrame
    df_max_analysis = pd.DataFrame(daily_maxes)
    
    if df_max_analysis.empty:
        print("No matching data found for analysis")
        return None
    
    # Create visualizations
    import plotly.express as px
    import plotly.graph_objects as go
    
    # 1. Scatter plot of forecast vs actual max temps
    fig1 = px.scatter(df_max_analysis,
                     x='actual_max_temp',
                     y='forecast_temp',
                     hover_data={
                         'date': True,
                         'hours_ahead': ':.2f',
                         'error': ':.2f',
                         'forecast_created': True,
                         'actual_max_temp': ':.2f',
                         'forecast_temp': ':.2f'
                     },
                     title='Forecast vs Actual Daily Maximum Temperatures')
    
    # Add perfect forecast line
    fig1.add_trace(go.Scatter(
        x=[df_max_analysis['actual_max_temp'].min(),
           df_max_analysis['actual_max_temp'].max()],
        y=[df_max_analysis['actual_max_temp'].min(),
           df_max_analysis['actual_max_temp'].max()],
        mode='lines',
        name='Perfect Forecast',
        line=dict(dash='dash')))
    fig1.show()
    
    # 2. Error distribution by hours ahead
    fig2 = px.box(df_max_analysis,
                  x=df_max_analysis['hours_ahead'].round(),
                  y='error',
                  hover_data=['date', 'forecast_created'],
                  title='Forecast Error Distribution by Lead Time (Daily Max Temp)')
    fig2.show()
    
    # 3. Time series of errors
    fig3 = px.scatter(df_max_analysis.sort_values('date'),
                   x='date',
                   y='error',
                   hover_data={
                       'hours_ahead': ':.2f',
                       'forecast_created': True,
                       'actual_max_temp': ':.2f',
                       'forecast_temp': ':.2f',
                       'error': ':.2f'
                   },
                   title='Daily Maximum Temperature Forecast Errors Over Time')
    fig3.add_hline(y=0, line_dash="dash", line_color="red")
    fig3.show()
    
    # Print summary statistics
    print("\nSummary Statistics for Daily Maximum Temperature Forecasts:")
    print(f"Mean Absolute Error: {df_max_analysis['error_magnitude'].mean():.2f}°F")
    print(f"Median Absolute Error: {df_max_analysis['error_magnitude'].median():.2f}°F")
    print(f"Number of forecasts analyzed: {len(df_max_analysis)}")
    print(f"Number of unique days: {df_max_analysis['date'].nunique()}")
    
    # Additional statistics by forecast horizon
    print("\nError by Forecast Horizon:")
    horizon_stats = df_max_analysis.groupby(
        df_max_analysis['hours_ahead'].round()
    )['error_magnitude'].agg(['mean', 'count']).round(2)
    print(horizon_stats)
    
    return df_max_analysis

# Run the analysis
df_max_analysis = analyze_daily_max_temps(df_forecasts, df_observations)


Summary Statistics for Daily Maximum Temperature Forecasts:
Mean Absolute Error: 6.48°F
Median Absolute Error: 7.08°F
Number of forecasts analyzed: 10
Number of unique days: 1

Error by Forecast Horizon:
             mean  count
hours_ahead             
1.0          5.08      1
3.0          5.08      1
4.0          5.08      1
5.0          7.08      1
6.0          7.08      1
7.0          7.08      1
8.0          7.08      1
9.0          7.08      1
10.0         7.08      1
11.0         7.08      1


In [11]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def analyze_max_temp_forecast_accuracy(df_forecasts, df_observations, temp_margin=1.0):
    """
    For each daily high:
    1. Look at forecast groups made within 12 hours before the high
    2. For each group, find the maximum temperature forecast that's:
       - Predicting a time before the high (but within 12 hours)
       - Not predicting after the high's time
    """
    daily_maxes = []
    df_observations['date'] = df_observations['observed_time'].dt.date
    
    # First find daily highs
    for date, day_obs in df_observations.groupby('date'):
        # Get max temp and its exact time for this day
        max_temp = day_obs['temperature'].max()
        max_temp_obs = day_obs[day_obs['temperature'] == max_temp].iloc[-1]  # Take latest if multiple
        high_time = max_temp_obs['observed_time']
        
        # Look at forecasts created within 12 hours before the high
        forecast_window_start = high_time - timedelta(hours=12)
        
        # Group forecasts by their creation time
        for created_at, group in df_forecasts[
            (df_forecasts['created_at'] >= forecast_window_start) &
            (df_forecasts['created_at'] <= high_time)
        ].groupby('created_at'):
            
            # Filter for valid forecast times and get the max temperature
            valid_forecasts = group[
                (group['forecast_time'] <= high_time) &
                (group['forecast_time'] >= high_time - timedelta(hours=12))
            ]
            
            if not valid_forecasts.empty:
                # Get the maximum temperature forecast from this group
                max_forecast = valid_forecasts.loc[valid_forecasts['temperature'].idxmax()]
                error = max_forecast['temperature'] - max_temp
                is_accurate = abs(error) <= temp_margin
                hours_before_high = (high_time - max_forecast['forecast_time']).total_seconds() / 3600
                
                daily_maxes.append({
                    'date': date,
                    'high_temp': max_temp,
                    'high_temp_time': high_time,
                    'forecast_max_temp': max_forecast['temperature'],
                    'forecast_time': max_forecast['forecast_time'],
                    'forecast_created': created_at,
                    'hours_before_high': hours_before_high,  # How far before the high this forecast was for
                    'hours_ahead': (max_forecast['forecast_time'] - created_at).total_seconds() / 3600,  # Lead time
                    'error': error,
                    'error_magnitude': abs(error),
                    'is_accurate': is_accurate
                })
    
    df_analysis = pd.DataFrame(daily_maxes)
    
        # ... (previous code remains the same until after df_analysis creation) ...
    
    if df_analysis.empty:
        print("No matching data found for analysis")
        return None
    
    # Print summary for each day (only once)
    print("\nDaily Maximum Temperature Analysis:")
    print("===================================")
    
    for date, day_data in df_analysis.groupby('date'):
        high_temp = day_data['high_temp'].iloc[0]
        high_time = day_data['high_temp_time'].iloc[0]
        accurate_forecasts = day_data[day_data['is_accurate']]
        
        print(f"\nDate: {date}")
        print(f"Daily High: {high_temp:.2f}°F at {high_time.strftime('%I:%M %p EST')}")
        print(f"Number of forecast groups: {len(day_data)}")
        print(f"Accurate max forecasts (±{temp_margin}°F): {len(accurate_forecasts)}")
        
        if len(accurate_forecasts) > 0:
            print("\nAccurate Forecasts:")
            for _, forecast in accurate_forecasts.iterrows():
                print(f"  Created: {forecast['forecast_created'].strftime('%Y-%m-%d %I:%M %p EST')}")
                print(f"  Predicted: {forecast['forecast_max_temp']:.2f}°F")
                print(f"  Hours ahead: {forecast['hours_ahead']:.1f}")
                print(f"  Error: {forecast['error']:.2f}°F")
                print("  ---")
    
    # 1. Scatter plot of all forecasts vs actual highs
    fig1 = px.scatter(df_analysis,
                     x='high_temp',
                     y='forecast_max_temp',
                     color='hours_ahead',
                     color_continuous_scale='RdYlBu_r',
                     range_color=[0, 14],
                     hover_data={
                         'date': True,
                         'high_temp_time': True,
                         'forecast_created': True,
                         'hours_ahead': ':.1f',
                         'error': ':.2f',
                         'high_temp': ':.2f',
                         'forecast_max_temp': ':.2f',
                         'is_accurate': True
                     },
                     title=f'Forecast vs Actual Daily Maximum Temperatures (±{temp_margin}°F margin)<br>Color shows hours before actual high')
    
    # Make points bigger
    fig1.update_traces(marker=dict(size=12))
    
    # Add perfect forecast line and margin bands
    temp_range = np.linspace(
        df_analysis['high_temp'].min() - 5,
        df_analysis['high_temp'].max() + 5,
        100
    )
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range,
        mode='lines',
        name='Perfect Forecast',
        line=dict(color='black', dash='dash')
    ))
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range + temp_margin,
        mode='lines',
        name=f'+{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range - temp_margin,
        mode='lines',
        name=f'-{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    
    fig1.show()
    
    # 2. Time series with color showing lead time
    fig2 = px.scatter(df_analysis,
                     x='date',
                     y='forecast_max_temp',  # Changed from forecast_temp
                     color='hours_ahead',
                     color_continuous_scale='RdYlBu_r',
                     range_color=[0, 14],
                     hover_data={
                         'forecast_time': True,
                         'forecast_created': True,
                         'hours_ahead': ':.1f',
                         'error': ':.2f',
                         'high_temp': ':.2f',
                         'forecast_max_temp': ':.2f',
                         'is_accurate': True
                     },
                     title='Daily Maximum Temperature Forecasts Over Time<br>Color shows hours before actual high')
    
    # Make points bigger
    fig2.update_traces(marker=dict(size=12))
    
    # Add actual max temperatures line
    daily_highs = df_analysis.groupby('date')['high_temp'].first().reset_index()
    fig2.add_trace(go.Scatter(
        x=daily_highs['date'],
        y=daily_highs['high_temp'],
        mode='lines+markers',
        name='Actual Daily Max',
        line=dict(color='black'),
        marker=dict(size=12)
    ))
    
    # Update x-axis to show only dates
    fig2.update_xaxes(
        tickformat="%Y-%m-%d",
        dtick="D1",  # Show tick for each day
        tickangle=45  # Angle the dates for better readability
    )
    
    fig2.show()
    return df_analysis

# Run the analysis
df_max_analysis = analyze_max_temp_forecast_accuracy(df_forecasts, df_observations, temp_margin=1.0)


Daily Maximum Temperature Analysis:

Date: 2024-12-09
Daily High: 51.08°F at 10:00 AM EST
Number of forecast groups: 10
Accurate max forecasts (±1.0°F): 0


In [29]:
def analyze_max_temp_forecast_accuracy(df_forecasts, df_observations, temp_margin=1.0, timezone='America/New_York'):
    """
    For each daily high temperature:
    1. Look back 12 hours before the high
    2. For each hour in that window:
        - Get all forecasts created in that hour
        - Filter for forecasts predicting temps for the same local day
        - Take the maximum forecasted temperature
    """
    # Convert times to local timezone
    df_observations = df_observations.copy()
    df_forecasts = df_forecasts.copy()
    
    df_observations['local_time'] = df_observations['observed_time'].dt.tz_convert(timezone)
    df_observations['local_date'] = df_observations['local_time'].dt.date
    
    df_forecasts['local_created'] = df_forecasts['created_at'].dt.tz_convert(timezone)
    df_forecasts['local_forecast_time'] = df_forecasts['forecast_time'].dt.tz_convert(timezone)
    df_forecasts['local_date'] = df_forecasts['local_forecast_time'].dt.date
    
    daily_maxes = []
    
    # Find daily highs in local time
    for date, day_obs in df_observations.groupby('local_date'):
        max_temp = day_obs['temperature'].max()
        max_temp_obs = day_obs[day_obs['temperature'] == max_temp].iloc[-1]
        high_local_time = max_temp_obs['local_time']
        
        # Look back 12 hours before the high
        window_start = high_local_time - pd.Timedelta(hours=12)
        
        # For each hour in that window
        current_hour = window_start.replace(minute=0, second=0, microsecond=0)
        hour_count = 0
        while current_hour <= high_local_time:
            hour_forecasts = df_forecasts[
                (df_forecasts['local_created'] >= current_hour) &
                (df_forecasts['local_created'] < current_hour + pd.Timedelta(minutes=2)) &
                (df_forecasts['local_date'] == date)
            ]
            
            if not hour_forecasts.empty:
                max_forecast = hour_forecasts.loc[hour_forecasts['temperature'].idxmax()]
                error = max_forecast['temperature'] - max_temp
                is_accurate = abs(error) <= temp_margin
                hours_until_high = (high_local_time - current_hour).total_seconds() / 3600
                
                daily_maxes.append({
                    'date': date,
                    'high_temp': max_temp,
                    'high_temp_time': high_local_time,
                    'forecast_max_temp': max_forecast['temperature'],
                    'forecast_time': max_forecast['local_forecast_time'],
                    'forecast_created': current_hour,
                    'hours_until_high': hours_until_high,
                    'num_forecasts': len(hour_forecasts),
                    'error': error,
                    'error_magnitude': abs(error),
                    'is_accurate': is_accurate,
                    'hour_number': hour_count
                })
            hour_count += 1
            current_hour += pd.Timedelta(hours=1)
    
    df_analysis = pd.DataFrame(daily_maxes)
    
    if df_analysis.empty:
        print("No matching data found for analysis")
        return None

    # Print summary
    print("\nDaily Maximum Temperature Analysis:")
    print("===================================")
    
    for date, day_data in df_analysis.groupby('date'):
        high_temp = day_data['high_temp'].iloc[0]
        high_time = day_data['high_temp_time'].iloc[0]
        accurate_forecasts = day_data[day_data['is_accurate']]
        
        print(f"\nDate: {date}")
        print(f"Daily High: {high_temp:.2f}°F at {high_time.strftime('%I:%M %p')}")
        print(f"Number of forecast groups: {len(day_data)}")
        print(f"Accurate max forecasts (±{temp_margin}°F): {len(accurate_forecasts)}")
        
        if len(accurate_forecasts) > 0:
            print("\nAccurate Forecasts:")
            for _, forecast in accurate_forecasts.iterrows():
                print(f"  Created: {forecast['forecast_created'].strftime('%I:%M %p')}")
                print(f"  Predicted: {forecast['forecast_max_temp']:.2f}°F")
                print(f"  Hours until high: {forecast['hours_until_high']:.1f}")
                print(f"  Error: {forecast['error']:.2f}°F")
                print(f"  Number of forecasts: {forecast['num_forecasts']}")
                print("  ---")
    
    # 1. Scatter plot with fixed hover template
    fig1 = px.scatter(df_analysis,
                     x='high_temp',
                     y='forecast_max_temp',
                     size='hours_until_high',
                     size_max=75,  # Increased to 125
                     color='hours_until_high',
                     color_continuous_scale='RdYlBu_r',
                     range_color=[0, 12],
                     hover_data={
                         'date': True,
                         'forecast_created': True,
                         'num_forecasts': True,
                         'hours_until_high': ':.1f',
                         'error': ':.2f',
                         'high_temp': ':.2f',
                         'forecast_max_temp': ':.2f',
                         'is_accurate': True
                     },
                     title='Maximum Temperature Forecasts vs Actual Daily Highs<br>Size and color show hours until high temperature')
    
    # Customize hover template for fig1
    fig1.update_traces(
        hovertemplate=(
            "Date: %{customdata[0]}<br>" +
            "Created: %{customdata[1]}<br>" +
            "Forecasts in hour: %{customdata[2]}<br>" +
            "Hours until high: %{customdata[3]:.1f}<br>" +
            "Error: %{customdata[4]:.2f}°F<br>" +
            "High Temp: %{x:.2f}°F<br>" +  # Changed to use x coordinate
            "Forecast Max: %{y:.2f}°F<br>" +
            "Accurate: %{customdata[5]}" +  # Changed index
            "<extra></extra>"
        )
    )
    
    # Add perfect forecast line and margin bands
    temp_range = np.linspace(
        df_analysis['high_temp'].min() - 5,
        df_analysis['high_temp'].max() + 5,
        100
    )
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range,
        mode='lines',
        name='Perfect Forecast',
        line=dict(color='black', dash='dash')
    ))
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range + temp_margin,
        mode='lines',
        name=f'+{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    
    fig1.add_trace(go.Scatter(
        x=temp_range,
        y=temp_range - temp_margin,
        mode='lines',
        name=f'-{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    
    fig1.show()
    
     # 2. Time series with fixed hover template
    fig2 = px.scatter(df_analysis,
                     x='date',
                     y='forecast_max_temp',
                     size='hours_until_high',
                     size_max=75,
                     color='hours_until_high',
                     color_continuous_scale='RdYlBu_r',
                     range_color=[0, 12],
                     hover_data={
                         'forecast_created': True,
                         'num_forecasts': True,
                         'hours_until_high': ':.1f',
                         'error': ':.2f',
                         'high_temp': ':.2f',
                         'forecast_max_temp': ':.2f',
                         'is_accurate': True
                     },
                     title='Daily Maximum Temperature Forecasts Over Time<br>Size and color show hours until high temperature')
    
    # Add actual max temperatures line
    daily_highs = df_analysis.groupby('date')['high_temp'].first().reset_index()
    
    # Perfect forecast line (actual temperatures)
    fig2.add_trace(go.Scatter(
        x=daily_highs['date'],
        y=daily_highs['high_temp'],
        mode='lines+markers',
        name='Actual Daily Max',
        line=dict(color='black', dash='solid'),
        marker=dict(size=12)
    ))
    
    # Add +1°F margin line
    fig2.add_trace(go.Scatter(
        x=daily_highs['date'],
        y=daily_highs['high_temp'] + temp_margin,
        mode='lines',
        name=f'+{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    
    # Add -1°F margin line
    fig2.add_trace(go.Scatter(
        x=daily_highs['date'],
        y=daily_highs['high_temp'] - temp_margin,
        mode='lines',
        name=f'-{temp_margin}°F Margin',
        line=dict(color='gray', dash='dot')
    ))
    # Update x-axis to show only dates
    fig2.update_xaxes(
        tickformat="%Y-%m-%d",
        dtick="D1",
        tickangle=45
    )
    
    fig2.show()
    return df_analysis

# Run the analysis
df_max_analysis = analyze_max_temp_forecast_accuracy(df_forecasts, df_observations, temp_margin=1.0)


Daily Maximum Temperature Analysis:

Date: 2024-12-09
Daily High: 51.08°F at 10:00 AM
Number of forecast groups: 11
Accurate max forecasts (±1.0°F): 0
